In [5]:
![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
import matplotlib.pyplot as plt

# Nettoyage des données
history['close'] = history['close'].apply(lambda x: x[0] if isinstance(x, (list, tuple)) else x)


# Visualisation
plt.figure(figsize=(14, 7))
plt.plot(history.index, history['close'], label='Prix', linewidth=1)
plt.plot(history.index, history['SMA'], label='Moyenne mobile', linewidth=1)
plt.plot(history.index, history['UpperBand'], label='Bande supérieure', linestyle='--')
plt.plot(history.index, history['LowerBand'], label='Bande inférieure', linestyle='--')
plt.legend()
plt.title('Bandes de Bollinger sur BTCUSD')
plt.xlabel('Date')
plt.ylabel('Prix')
plt.grid()
plt.show()


In [ ]:
# Import des bibliothèques nécessaires pour QuantConnect
from AlgorithmImports import *
import matplotlib.pyplot as plt

# Définition de la classe de l'algorithme
class CryptoMovingAverageCrossAlgorithm(QCAlgorithm):

    def Initialize(self):
        # Configuration Initiale
        self.SetStartDate(2023, 1, 1)  # Date de début
        self.SetEndDate(2025, 1, 1)    # Date de fin
        self.SetCash(200000)           # Capital initial
        self.SetAccountCurrency("USDT") # Définir la devise du compte en USDT

        # Configuration de Binance et des Cryptomonnaies
        self.SetBrokerageModel(BrokerageName.Binance, AccountType.Cash)
        self.btc_symbol = self.AddCrypto("BTCUSDT", Resolution.Daily, Market.Binance).Symbol

        # Indicateurs des Moyennes Mobiles
        self.ma5 = self.SMA(self.btc_symbol, 5, Resolution.Daily)  # Moyenne mobile à 5 jours
        self.ma20 = self.SMA(self.btc_symbol, 20, Resolution.Daily) # Moyenne mobile à 20 jours
        
        # Benchmark
        self.SetBenchmark(self.btc_symbol)

        # Suivi des positions
        self.current_position = None  # Variable pour stocker l'état actuel (achat ou vente)

    def OnData(self, data):
        # Vérifier si tous les indicateurs sont prêts
        if not self.ma5.IsReady or not self.ma20.IsReady:
            return

        # Obtenir les valeurs actuelles de MA5 et MA20
        ma5_value = self.ma5.Current.Value
        ma20_value = self.ma20.Current.Value
        price = data[self.btc_symbol].Price  # Prix actuel du Bitcoin

        # Logique d'achat
        if self.current_position != "long" and ma5_value > ma20_value:
            self.SetHoldings(self.btc_symbol, 1.0)  # Allouer 100% du capital au Bitcoin
            self.current_position = "long"  # Marquer la position comme "long"
            self.Debug(f"Achat BTCUSDT à {price} ; MA5: {ma5_value}, MA20: {ma20_value}")

        # Logique de vente
        elif self.current_position != "short" and ma5_value < ma20_value:
            self.Liquidate(self.btc_symbol)  # Vendre toutes les positions
            self.current_position = "short"  # Marquer la position comme "short"
            self.Debug(f"Vente BTCUSDT à {price} ; MA5: {ma5_value}, MA20: {ma20_value}")

    def OnEndOfAlgorithm(self):
        # Récupérer les données historiques pour la visualisation
        history = self.History(self.btc_symbol, 100, Resolution.Daily)
        history['SMA'] = history['close'].rolling(window=20).mean()
        history['UpperBand'] = history['SMA'] + 2 * history['close'].rolling(window=20).std()
        history['LowerBand'] = history['SMA'] - 2 * history['close'].rolling(window=20).std()

        # Nettoyage des données
        history['close'] = history['close'].apply(lambda x: x[0] if isinstance(x, (list, tuple)) else x)

        # Visualisation
        plt.figure(figsize=(14, 7))
        plt.plot(history.index, history['close'], label='Prix', linewidth=1)
        plt.plot(history.index, history['SMA'], label='Moyenne mobile', linewidth=1)
        plt.plot(history.index, history['UpperBand'], label='Bande supérieure', linestyle='--')
        plt.plot(history.index, history['LowerBand'], label='Bande inférieure', linestyle='--')
        plt.legend()
        plt.title('Bandes de Bollinger sur BTCUSD')
        plt.xlabel('Date')
        plt.ylabel('Prix')
        plt.grid()
        plt.show()

# Pour exécuter ce code, assurez-vous d'utiliser un environnement compatible avec QuantConnect Lean.